In [1]:
import pandas as pd
import pickle

#Para el área
from shapely.geometry import Polygon
from shapely.geometry import MultiPoint 
from scipy.spatial import ConvexHull
from shapely.geometry import LineString
from shapely.geometry.point import Point

In [2]:
pd.set_option('display.max_columns',None)

In [3]:
week1 = pd.read_csv("../processed_data/clean_without_pass_rushers/week1.csv")
week2 = pd.read_csv("../processed_data/clean_without_pass_rushers/week2.csv")
week3 = pd.read_csv("../processed_data/clean_without_pass_rushers/week3.csv")
week4 = pd.read_csv("../processed_data/clean_without_pass_rushers/week4.csv")
week5 = pd.read_csv("../processed_data/clean_without_pass_rushers/week5.csv")
week6 = pd.read_csv("../processed_data/clean_without_pass_rushers/week6.csv")
week7 = pd.read_csv("../processed_data/clean_without_pass_rushers/week7.csv")
week8 = pd.read_csv("../processed_data/clean_without_pass_rushers/week8.csv")
week9 = pd.read_csv("../processed_data/clean_without_pass_rushers/week9.csv")
week10 = pd.read_csv("../processed_data/clean_without_pass_rushers/week10.csv")
week11 = pd.read_csv("../processed_data/clean_without_pass_rushers/week11.csv")
week12 = pd.read_csv("../processed_data/clean_without_pass_rushers/week12.csv")
week13 = pd.read_csv("../processed_data/clean_without_pass_rushers/week13.csv")
week14 = pd.read_csv("../processed_data/clean_without_pass_rushers/week14.csv")
week15 = pd.read_csv("../processed_data/clean_without_pass_rushers/week15.csv")
week16 = pd.read_csv("../processed_data/clean_without_pass_rushers/week16.csv")
week17 = pd.read_csv("../processed_data/clean_without_pass_rushers/week17.csv")
week1.head()

,Unnamed: 0,id,time,x,y,s,a,dis,o,dir,event,nflId,displayName,jerseyNumber,position,frameId,team,playDirection,route
0,0,2018090600:75:1,2018-09-07T01:07:14.599Z,91.73,26.67,0.00,0.01,0.02,289.57,240.93,None,310.0,Matt Ryan,2.0,QB,1,away,left,NaN
1,1,2018090600:75:1,2018-09-07T01:07:14.599Z,88.89,36.47,0.01,0.01,0.01,105.63,66.66,None,79848.0,Malcolm Jenkins,27.0,SS,1,home,left,NaN
2,2,2018090600:75:1,2018-09-07T01:07:14.599Z,91.35,44.16,0.02,0.03,0.01,290.45,16.86,None,2495454.0,Julio Jones,11.0,WR,1,away,left,HITCH
3,3,2018090600:75:1,2018-09-07T01:07:14.599Z,86.31,22.01,0.09,0.42,0.01,70.12,168.91,None,2495613.0,Corey Graham,24.0,FS,1,home,left,NaN
4,4,2018090600:75:1,2018-09-07T01:07:14.599Z,90.78,36.15,0.00,0.00,0.00,257.61,193.97,None,2533040.0,Mohamed Sanu,12.0,WR,1,away,left,HITCH


In [4]:
weeks = [week1,week2,week3,week4,week5,week6,week7,week8,week9,week10,week11,week12,week13,week14,week15,week16,week17]

In [5]:
for week in weeks:
    week.drop(["Unnamed: 0"],axis=1,inplace=True)

In [6]:
static_defense = pd.read_csv("../transformed_data/static_defenses_yards.csv",index_col=1)
static_defense.head()

,Unnamed: 0,defensivelinezonePlayers,deepzonePlayers,hookzonePlayers,curlzonePlayers,flatzonePlayers,defenseArea,defenseAreaCoverDefenders,width,height,numberQBs,numberWRs,numberTEs,numberRBs,numberFBs,numberOffensivePlayersAnotherPosition,numberSafeties,numberLBs,numberCBs,strongSide,numberPlayersDefenseStrongSide,numberPlayersDefenseWeakSide,numberPlayersOffenseStrongSide,numberPlayersOffenseWeakSide,differenceOffenseVsDefenseWidth,differenceOffenseVsDefenseStrongSide,differenceOffenseVsDefenseWeakSide,HeightByWeightDeep,HeightByWeightHook,HeightByWeightCurl,HeightByWeightFlat,WeightByArea,density,densityNoLine,densityInsidePoints,densityInsidePointsNoLine,playResult
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2018090600:75:1,0,0,1,3,2,1,194.31095,194.31095,22.96,15.25,1,2,1,1,1,0,3,2,2,left,3,4,3,2,-0.63,0,-2,0.020227,0.018643,0.020066,0.021034,3.376032,0.036025,0.036025,6.974268,6.974268,10
2018090600:146:1,1,0,1,3,2,1,204.17725,204.17725,29.22,12.57,1,3,1,1,0,0,2,2,3,right,3,4,2,3,-1.15,-1,-1,0.020227,0.019228,0.020356,0.020455,3.178611,0.034284,0.034284,6.975511,6.975511,0
2018090600:168:1,2,0,1,2,2,2,277.65495,277.65495,33.67,15.21,1,2,1,2,0,0,2,2,3,left,5,2,4,1,0.71,-1,-1,0.020227,0.017683,0.020997,0.020745,2.337434,0.025211,0.025211,6.985594,6.985594,0
2018090600:190:1,3,1,0,2,3,1,189.16960,165.05520,26.42,12.91,1,3,1,1,0,0,3,1,3,right,2,5,2,3,-3.11,0,-2,0.000000,0.020120,0.021193,0.020455,3.192871,0.037004,0.036351,6.973569,6.969707,33
2018090600:320:1,5,1,0,3,3,1,187.12510,153.20020,27.80,13.29,1,3,1,1,0,0,2,3,3,left,3,5,3,2,-4.82,0,-3,0.000000,0.018995,0.019537,0.021279,4.281979,0.042752,0.045692,7.978624,7.980418,4


In [7]:
static_defense.drop(["Unnamed: 0"],axis=1,inplace=True)

In [8]:
plays = pd.read_csv("../processed_data/clean/plays.csv",index_col=1)
plays.head()

,Unnamed: 0,playDescription,quarter,down,yardsToGo,possessionTeam,playType,yardlineSide,yardlineNumber,offenseFormation,personnelO,defendersInTheBox,numberOfPassRushers,personnelD,preSnapVisitorScore,preSnapHomeScore,gameClock,absoluteYardlineNumber,penaltyCodes,penaltyJerseyNumbers,passResult,offensePlayResult,playResult,epa,isDefensivePI
id,,,,,,,,,,,,,,,,,,,,,,,,,
2018090600:75:1,0,(15:00) M.Ryan pass short right to J.Jones pus...,1,1,15,ATL,play_type_pass,ATL,20,I_FORM,"2 RB, 1 TE, 2 WR",7.0,4.0,"4 DL, 2 LB, 5 DB",0.0,0.0,15:00:00,90.0,NaN,NaN,C,10,10,0.261827,False
2018090600:146:1,1,(13:10) M.Ryan pass incomplete short right to ...,1,1,10,ATL,play_type_pass,PHI,39,SINGLEBACK,"1 RB, 1 TE, 3 WR",7.0,4.0,"4 DL, 2 LB, 5 DB",0.0,0.0,13:10:00,49.0,NaN,NaN,I,0,0,-0.372360,False
2018090600:168:1,2,(13:05) (Shotgun) M.Ryan pass incomplete short...,1,2,10,ATL,play_type_pass,PHI,39,SHOTGUN,"2 RB, 1 TE, 2 WR",6.0,4.0,"4 DL, 2 LB, 5 DB",0.0,0.0,13:05:00,49.0,NaN,NaN,I,0,0,-0.702779,False
2018090600:190:1,3,(13:01) (Shotgun) M.Ryan pass deep left to J.J...,1,3,10,ATL,play_type_pass,PHI,39,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,5.0,"4 DL, 1 LB, 6 DB",0.0,0.0,13:01:00,49.0,NaN,NaN,C,33,33,3.047530,False
2018090600:256:1,4,(10:59) (Shotgun) M.Ryan pass incomplete short...,1,3,1,ATL,play_type_pass,PHI,1,SHOTGUN,"2 RB, 3 TE, 0 WR",8.0,6.0,"6 DL, 3 LB, 2 DB",0.0,0.0,10:59:00,11.0,NaN,NaN,I,0,0,-0.842272,False


In [9]:
plays.drop(["Unnamed: 0"],axis=1,inplace=True)

In [10]:
def obtainDefenseTeam(playId):
    playIdParts = playId.split(":")
    gameId = playIdParts[0]
    visitorTeam = games.loc[int(gameId)]["visitorTeamAbbr"]
    homeTeam = games.loc[int(gameId)]["homeTeamAbbr"]
    possessionTeam = plays.loc[playId]["possessionTeam"]
    
    if possessionTeam == visitorTeam: #si equipo visitante tiene el balón, el equipo local es el que defiende
        value = "home"
    elif possessionTeam == homeTeam:
        value = "away"
    
    return value

In [11]:
def obtainOffenseTeam(playId):
    playIdParts = playId.split(":")
    gameId = playIdParts[0]
    visitorTeam = games.loc[int(gameId)]["visitorTeamAbbr"]
    homeTeam = games.loc[int(gameId)]["homeTeamAbbr"]
    possessionTeam = plays.loc[playId]["possessionTeam"]
    
    if possessionTeam == visitorTeam: #si equipo visitante tiene el balón, el equipo local es el que defiende
        value = "away"
    elif possessionTeam == homeTeam:
        value = "home"
    
    return value

In [12]:
games = pd.read_csv("../raw_data/games.csv",index_col=0)
games.head()

,gameDate,gameTimeEastern,homeTeamAbbr,visitorTeamAbbr,week
gameId,,,,,
2018090600,09/06/2018,20:20:00,PHI,ATL,1
2018090901,09/09/2018,13:00:00,CLE,PIT,1
2018090902,09/09/2018,13:00:00,IND,CIN,1
2018090903,09/09/2018,13:00:00,MIA,TEN,1
2018090900,09/09/2018,13:00:00,BAL,BUF,1


In [13]:
summary = pd.DataFrame()

# Obtener todas las jugadas de tercer down

In [14]:
third_down_plays = plays[(plays["down"]==3)&(plays["passResult"]!="S")]
third_down_plays

,playDescription,quarter,down,yardsToGo,possessionTeam,playType,yardlineSide,yardlineNumber,offenseFormation,personnelO,defendersInTheBox,numberOfPassRushers,personnelD,preSnapVisitorScore,preSnapHomeScore,gameClock,absoluteYardlineNumber,penaltyCodes,penaltyJerseyNumbers,passResult,offensePlayResult,playResult,epa,isDefensivePI
id,,,,,,,,,,,,,,,,,,,,,,,,
2018090600:190:1,(13:01) (Shotgun) M.Ryan pass deep left to J.J...,1,3,10,ATL,play_type_pass,PHI,39,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,5.0,"4 DL, 1 LB, 6 DB",0.0,0.0,13:01:00,49.0,NaN,NaN,C,33,33,3.047530,False
2018090600:256:1,(10:59) (Shotgun) M.Ryan pass incomplete short...,1,3,1,ATL,play_type_pass,PHI,1,SHOTGUN,"2 RB, 3 TE, 0 WR",8.0,6.0,"6 DL, 3 LB, 2 DB",0.0,0.0,10:59:00,11.0,NaN,NaN,I,0,0,-0.842272,False
2018090600:344:1,(9:24) (Shotgun) N.Foles pass incomplete short...,1,3,4,PHI,play_type_pass,PHI,8,EMPTY,"1 RB, 2 TE, 2 WR",6.0,4.0,"3 DL, 3 LB, 5 DB",0.0,0.0,09:24:00,18.0,NaN,NaN,I,0,0,-1.192208,False
2018090600:636:1,(5:07) (Shotgun) M.Ryan pass incomplete short ...,1,3,3,ATL,play_type_pass,PHI,3,SHOTGUN,"1 RB, 2 TE, 2 WR",6.0,4.0,"4 DL, 2 LB, 5 DB",0.0,0.0,05:07:00,13.0,NaN,NaN,I,0,0,-0.874640,False
2018090600:949:1,(15:00) (Shotgun) M.Ryan pass short left to D....,2,3,12,ATL,play_type_pass,ATL,18,SHOTGUN,"1 RB, 1 TE, 3 WR",4.0,4.0,"4 DL, 1 LB, 6 DB",3.0,0.0,15:00:00,28.0,NaN,NaN,C,-7,-7,-1.912483,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018123015:2373:17,(11:58) (Shotgun) J.Rosen pass incomplete shor...,3,3,2,ARI,play_type_pass,ARI,27,EMPTY,"1 RB, 1 TE, 3 WR",5.0,4.0,"4 DL, 2 LB, 5 DB",13.0,14.0,11:58:00,37.0,NaN,NaN,I,0,0,-1.659557,False
2018123015:2629:17,(7:43) (Shotgun) J.Rosen pass incomplete deep ...,3,3,13,ARI,play_type_pass,SEA,48,SHOTGUN,"1 RB, 1 TE, 3 WR",4.0,4.0,"4 DL, 1 LB, 6 DB",13.0,14.0,07:43:00,62.0,NaN,NaN,I,0,0,-1.748119,False
2018123015:2998:17,(4:17) (Shotgun) J.Rosen pass short middle to ...,3,3,10,ARI,play_type_pass,ARI,30,SHOTGUN,"1 RB, 1 TE, 3 WR",5.0,4.0,"4 DL, 1 LB, 6 DB",13.0,21.0,04:17:00,40.0,NaN,NaN,C,9,9,-1.153889,False


In [15]:
third_down_plays = third_down_plays.sample(frac=0.025)
third_down_plays

,playDescription,quarter,down,yardsToGo,possessionTeam,playType,yardlineSide,yardlineNumber,offenseFormation,personnelO,defendersInTheBox,numberOfPassRushers,personnelD,preSnapVisitorScore,preSnapHomeScore,gameClock,absoluteYardlineNumber,penaltyCodes,penaltyJerseyNumbers,passResult,offensePlayResult,playResult,epa,isDefensivePI
id,,,,,,,,,,,,,,,,,,,,,,,,
2018093006:2225:4,(:07) (Shotgun) R.Tannehill pass short middle ...,2,3,1,MIA,play_type_pass,MIA,34,SHOTGUN,"1 RB, 0 TE, 4 WR",3.0,3.0,"3 DL, 1 LB, 7 DB",0.0,24.0,00:07:00,76.0,NaN,NaN,C,13,13,-1.771472,False
2018101500:3701:6,(1:13) (Shotgun) C.Beathard pass deep middle i...,4,3,3,SF,play_type_pass,GB,46,SHOTGUN,"1 RB, 1 TE, 3 WR",8.0,6.0,"2 DL, 3 LB, 6 DB",30.0,30.0,01:13:00,56.0,NaN,NaN,IN,0,0,-2.019912,False
2018122300:3525:16,(7:19) (Shotgun) A.Luck pass short middle to T...,4,3,5,IND,play_type_pass,IND,41,SHOTGUN,"1 RB, 0 TE, 4 WR",6.0,3.0,"1 DL, 5 LB, 5 DB",27.0,21.0,07:19:00,51.0,NaN,NaN,C,20,20,2.502640,False
2018121700:852:15,(:57) (Shotgun) C.Newton pass incomplete short...,1,3,1,CAR,play_type_pass,CAR,34,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,5.0,"4 DL, 2 LB, 5 DB",3.0,7.0,00:57:00,44.0,NaN,NaN,I,0,0,-1.694564,False
2018102500:273:8,(11:35) (Shotgun) B.Osweiler pass short left t...,1,3,11,MIA,play_type_pass,HOU,38,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,2.0,"2 DL, 4 LB, 5 DB",0.0,0.0,11:35:00,72.0,NaN,NaN,C,10,10,-0.348880,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018100100:2596:4,(6:55) (Shotgun) P.Mahomes pass incomplete dee...,3,3,17,KC,play_type_pass,DEN,44,SHOTGUN,"1 RB, 2 TE, 2 WR",5.0,4.0,"3 DL, 2 LB, 6 DB",13.0,13.0,06:55:00,54.0,NaN,NaN,I,0,0,-1.664076,False
2018110400:1702:9,(3:50) (Shotgun) B.Roethlisberger pass short r...,2,3,2,PIT,play_type_pass,PIT,33,SHOTGUN,"1 RB, 1 TE, 3 WR",7.0,6.0,"2 DL, 4 LB, 5 DB",14.0,6.0,03:50:00,77.0,NaN,NaN,C,5,5,1.310219,False
2018120202:4144:13,(3:07) (Shotgun) M.Stafford pass incomplete de...,4,3,12,DET,play_type_pass,LA,17,EMPTY,"0 RB, 1 TE, 4 WR",5.0,4.0,"4 DL, 1 LB, 6 DB",23.0,13.0,03:07:00,27.0,NaN,NaN,I,0,0,-0.917791,False


In [16]:
def evaluateDefense(playId):
    yardsToGo = third_down_plays.loc[playId]["yardsToGo"]
    offensePlayResult = third_down_plays.loc[playId]["offensePlayResult"]
    
    if offensePlayResult < yardsToGo:
        value = 1   #defensa exitosa
    else:
        value = 0   #mala defensa
        
    return value

In [17]:
third_down_plays["defenseValoration"] = third_down_plays.index.map(evaluateDefense)
third_down_plays.head()

,playDescription,quarter,down,yardsToGo,possessionTeam,playType,yardlineSide,yardlineNumber,offenseFormation,personnelO,defendersInTheBox,numberOfPassRushers,personnelD,preSnapVisitorScore,preSnapHomeScore,gameClock,absoluteYardlineNumber,penaltyCodes,penaltyJerseyNumbers,passResult,offensePlayResult,playResult,epa,isDefensivePI,defenseValoration
id,,,,,,,,,,,,,,,,,,,,,,,,,
2018093006:2225:4,(:07) (Shotgun) R.Tannehill pass short middle ...,2,3,1,MIA,play_type_pass,MIA,34,SHOTGUN,"1 RB, 0 TE, 4 WR",3.0,3.0,"3 DL, 1 LB, 7 DB",0.0,24.0,00:07:00,76.0,NaN,NaN,C,13,13,-1.771472,False,0
2018101500:3701:6,(1:13) (Shotgun) C.Beathard pass deep middle i...,4,3,3,SF,play_type_pass,GB,46,SHOTGUN,"1 RB, 1 TE, 3 WR",8.0,6.0,"2 DL, 3 LB, 6 DB",30.0,30.0,01:13:00,56.0,NaN,NaN,IN,0,0,-2.019912,False,1
2018122300:3525:16,(7:19) (Shotgun) A.Luck pass short middle to T...,4,3,5,IND,play_type_pass,IND,41,SHOTGUN,"1 RB, 0 TE, 4 WR",6.0,3.0,"1 DL, 5 LB, 5 DB",27.0,21.0,07:19:00,51.0,NaN,NaN,C,20,20,2.502640,False,0
2018121700:852:15,(:57) (Shotgun) C.Newton pass incomplete short...,1,3,1,CAR,play_type_pass,CAR,34,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,5.0,"4 DL, 2 LB, 5 DB",3.0,7.0,00:57:00,44.0,NaN,NaN,I,0,0,-1.694564,False,1
2018102500:273:8,(11:35) (Shotgun) B.Osweiler pass short left t...,1,3,11,MIA,play_type_pass,HOU,38,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,2.0,"2 DL, 4 LB, 5 DB",0.0,0.0,11:35:00,72.0,NaN,NaN,C,10,10,-0.348880,False,1


In [18]:
len(third_down_plays[third_down_plays["defenseValoration"]==0])

48

In [19]:
len(third_down_plays[third_down_plays["defenseValoration"]==1])

66

In [20]:
week1[(week1["id"]=="2018090900:1869:1")&(week1["frameId"]==1)&(week1["team"]==obtainDefenseTeam("2018090900:1869:1"))]

,id,time,x,y,s,a,dis,o,dir,event,nflId,displayName,jerseyNumber,position,frameId,team,playDirection,route
110087,2018090900:1869:1,2018-09-09T18:19:07.599Z,28.65,10.32,0.00,0.00,0.00,292.57,7.76,None,4365.0,Brandon Carr,24.0,CB,1,home,right,NaN
110090,2018090900:1869:1,2018-09-09T18:19:07.599Z,41.21,30.66,0.70,0.90,0.06,247.99,316.80,None,2495775.0,Eric Weddle,32.0,FS,1,home,right,NaN
110091,2018090900:1869:1,2018-09-09T18:19:07.599Z,33.82,18.26,0.45,0.75,0.04,295.26,35.27,None,2540164.0,Tony Jefferson,23.0,SS,1,home,right,NaN
110094,2018090900:1869:1,2018-09-09T18:19:07.599Z,28.55,18.95,0.34,0.66,0.03,269.93,230.71,None,2557841.0,Tim Williams,56.0,OLB,1,home,right,NaN
110096,2018090900:1869:1,2018-09-09T18:19:07.599Z,35.16,43.51,0.04,0.05,0.01,244.12,227.59,None,2558067.0,Marlon Humphrey,29.0,CB,1,home,right,NaN
110097,2018090900:1869:1,2018-09-09T18:19:07.599Z,32.46,23.12,0.54,1.74,0.05,265.73,331.39,None,2558166.0,Tyus Bowser,54.0,OLB,1,home,right,NaN


Tan solo están los jugadores que van a defender el pase

In [21]:
summary["playerId"] = ""

In [22]:
summary["playId"]=""

In [23]:
for play in third_down_plays.index:
    idParts = play.split(":")
    idWeek = idParts[2]
    week = weeks[int(idWeek)-1]
    #summary.append(week[(week["id"]==play)&(week["team"]==obtainDefenseTeam(play))]["nflId"].unique())
    for player in week[(week["id"]==play)&(week["team"]==obtainDefenseTeam(play))]["nflId"].unique():
        summary = summary.append({"playerId":player,"playId":play},ignore_index=True)

In [24]:
summary

,playerId,playId
0,71251.0,2018093006:2225:4
1,89756.0,2018093006:2225:4
2,494287.0,2018093006:2225:4
3,2533062.0,2018093006:2225:4
4,2541243.0,2018093006:2225:4
...,...,...
752,2550164.0,2018120902:699:14
753,2552389.0,2018120902:699:14
754,2555166.0,2018120902:699:14
755,2555540.0,2018120902:699:14


In [25]:
#save dataset
pickle_file = open('../resources/summary.pickle','wb')
pickle.dump(summary,pickle_file)
pickle_file.close()

In [28]:
#load dataset
pickle_file = open('../resources/summary.pickle','rb')
summary = pickle.load(pickle_file)
pickle_file.close()

Obtener los jugadores defensivos de esa jugada:


# Individual Actions Data (p): consideramos que el jugador ejecuta una acción en toda la jugada de la acción, hay que sacar por tanto una acción resumen de todas      para cada jugador p_i (i = a ... j) tenemos una serie de m características {p_i1, p_i2, ..., p_im}

In [18]:
#### Esto luego no servirá
week1_fpf = week1[week1["event"]=="pass_forward"]
week1_fpf.head()

,time,x,y,s,a,dis,o,dir,event,nflId,displayName,jerseyNumber,position,frameId,team,playDirection,route
id,,,,,,,,,,,,,,,,,
2018090600:75:1,2018-09-07T01:07:18.099Z,96.43,26.78,0.84,2.05,0.08,356.40,333.39,pass_forward,310.0,Matt Ryan,2.0,QB,36,away,left,NaN
2018090600:75:1,2018-09-07T01:07:18.099Z,84.82,38.33,5.53,4.30,0.54,295.34,318.58,pass_forward,79848.0,Malcolm Jenkins,27.0,SS,36,home,left,NaN
2018090600:75:1,2018-09-07T01:07:18.099Z,80.69,44.91,3.11,6.67,0.34,155.11,297.88,pass_forward,2495454.0,Julio Jones,11.0,WR,36,away,left,HITCH
2018090600:75:1,2018-09-07T01:07:18.099Z,86.02,18.64,3.52,0.69,0.35,28.19,216.15,pass_forward,2495613.0,Corey Graham,24.0,FS,36,home,left,NaN
2018090600:75:1,2018-09-07T01:07:18.099Z,82.65,34.56,6.35,1.21,0.64,262.06,272.78,pass_forward,2533040.0,Mohamed Sanu,12.0,WR,36,away,left,HITCH


In [30]:
defense_fpf = week1_fpf[week1_fpf.index.isin([selected_play]) & (week1_fpf["team"]==obtainDefenseTeam(selected_play))]
defense_fpf

,time,x,y,s,a,dis,o,dir,event,nflId,displayName,jerseyNumber,position,frameId,team,playDirection,route
id,,,,,,,,,,,,,,,,,
2018090905:313:1,2018-09-09T17:14:25.200Z,70.13,15.61,7.97,0.21,0.82,345.12,345.09,pass_forward,71251.0,Patrick Chung,23.0,SS,67,home,left,NaN
2018090905:313:1,2018-09-09T17:14:25.200Z,58.68,35.90,5.63,0.92,0.56,26.60,350.99,pass_forward,494287.0,Devin McCourty,32.0,FS,67,home,left,NaN
2018090905:313:1,2018-09-09T17:14:25.200Z,60.94,15.63,5.59,3.56,0.56,313.89,307.53,pass_forward,2533057.0,Dont'a Hightower,54.0,MLB,67,home,left,NaN
2018090905:313:1,2018-09-09T17:14:25.200Z,71.74,3.42,1.22,1.97,0.12,115.75,260.62,pass_forward,2533062.0,Stephon Gilmore,24.0,CB,67,home,left,NaN
2018090905:313:1,2018-09-09T17:14:25.200Z,60.70,12.93,4.42,3.73,0.44,12.95,340.10,pass_forward,2541243.0,Duron Harmon,21.0,FS,67,home,left,NaN
2018090905:313:1,2018-09-09T17:14:25.200Z,92.51,39.99,7.70,1.27,0.77,8.60,11.76,pass_forward,2543699.0,Kyle Van Noy,53.0,OLB,67,home,left,NaN
2018090905:313:1,2018-09-09T17:14:25.200Z,74.00,47.21,4.31,3.62,0.44,38.15,345.06,pass_forward,2552255.0,Eric Rowe,25.0,CB,67,home,left,NaN
2018090905:313:1,2018-09-09T17:14:25.200Z,68.22,51.49,3.01,2.70,0.31,349.57,298.95,pass_forward,2561143.0,Ja'Whaun Bentley,51.0,LB,67,home,left,NaN


In [23]:
week1[week1["event"].isin(["pass_forward"])]

,id,time,x,y,s,a,dis,o,dir,event,nflId,displayName,jerseyNumber,position,frameId,team,playDirection,route
490,2018090600:75:1,2018-09-07T01:07:18.099Z,96.43,26.78,0.84,2.05,0.08,356.40,333.39,pass_forward,310.0,Matt Ryan,2.0,QB,36,away,left,NaN
491,2018090600:75:1,2018-09-07T01:07:18.099Z,84.82,38.33,5.53,4.30,0.54,295.34,318.58,pass_forward,79848.0,Malcolm Jenkins,27.0,SS,36,home,left,NaN
492,2018090600:75:1,2018-09-07T01:07:18.099Z,80.69,44.91,3.11,6.67,0.34,155.11,297.88,pass_forward,2495454.0,Julio Jones,11.0,WR,36,away,left,HITCH
493,2018090600:75:1,2018-09-07T01:07:18.099Z,86.02,18.64,3.52,0.69,0.35,28.19,216.15,pass_forward,2495613.0,Corey Graham,24.0,FS,36,home,left,NaN
494,2018090600:75:1,2018-09-07T01:07:18.099Z,82.65,34.56,6.35,1.21,0.64,262.06,272.78,pass_forward,2533040.0,Mohamed Sanu,12.0,WR,36,away,left,HITCH
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
921155,2018091001:4192:1,2018-09-11T05:29:36.000Z,35.19,25.58,2.14,1.38,0.22,91.21,333.38,pass_forward,2556277.0,Marqui Christian,41.0,SS,26,away,left,NaN
921156,2018091001:4192:1,2018-09-11T05:29:36.000Z,35.85,38.20,2.68,1.33,0.26,79.60,343.60,pass_forward,2556593.0,Cory Littleton,58.0,ILB,26,away,left,NaN
921157,2018091001:4192:1,2018-09-11T05:29:36.000Z,40.85,46.18,0.92,4.07,0.08,156.73,134.27,pass_forward,2556885.0,Jalen Richard,30.0,RB,26,home,left,SCREEN
921158,2018091001:4192:1,2018-09-11T05:29:36.000Z,28.48,36.19,0.93,2.26,0.09,79.63,27.26,pass_forward,2558183.0,John Johnson,43.0,SS,26,away,left,NaN


In [20]:
week1.loc[selected_play]["event"].unique()

array(['None', 'ball_snap', 'pass_forward', 'pass_arrived',
       'pass_outcome_caught', 'out_of_bounds'], dtype=object)

In [21]:
week1_pa = week1[week1["event"]=="pass_arrived"]
week1_pa

,time,x,y,s,a,dis,o,dir,event,nflId,displayName,jerseyNumber,position,frameId,team,playDirection,route
id,,,,,,,,,,,,,,,,,
2018090600:75:1,2018-09-07T01:07:19.200Z,96.30,27.52,0.59,0.51,0.06,333.07,1.23,pass_arrived,310.0,Matt Ryan,2.0,QB,47,away,left,NaN
2018090600:75:1,2018-09-07T01:07:19.200Z,82.34,45.18,7.57,1.72,0.75,354.68,353.52,pass_arrived,79848.0,Malcolm Jenkins,27.0,SS,47,home,left,NaN
2018090600:75:1,2018-09-07T01:07:19.200Z,81.11,47.87,4.23,1.63,0.42,53.31,22.58,pass_arrived,2495454.0,Julio Jones,11.0,WR,47,away,left,HITCH
2018090600:75:1,2018-09-07T01:07:19.200Z,82.90,17.62,3.64,4.01,0.36,13.28,299.37,pass_arrived,2495613.0,Corey Graham,24.0,FS,47,home,left,NaN
2018090600:75:1,2018-09-07T01:07:19.200Z,77.66,35.52,2.99,3.53,0.31,78.88,305.48,pass_arrived,2533040.0,Mohamed Sanu,12.0,WR,47,away,left,HITCH
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018091001:4192:1,2018-09-11T05:29:36.799Z,34.72,28.43,4.96,3.17,0.48,44.27,359.68,pass_arrived,2556277.0,Marqui Christian,41.0,SS,34,away,left,NaN
2018091001:4192:1,2018-09-11T05:29:36.799Z,36.04,40.58,3.47,1.27,0.35,86.95,16.47,pass_arrived,2556593.0,Cory Littleton,58.0,ILB,34,away,left,NaN
2018091001:4192:1,2018-09-11T05:29:36.799Z,41.88,45.46,1.32,2.35,0.14,178.51,139.94,pass_arrived,2556885.0,Jalen Richard,30.0,RB,34,home,left,SCREEN


In [31]:
defense_pa = week1_pa[week1_pa.index.isin([selected_play]) & (week1_pa["team"]==obtainDefenseTeam(selected_play))]
defense_pa

,time,x,y,s,a,dis,o,dir,event,nflId,displayName,jerseyNumber,position,frameId,team,playDirection,route
id,,,,,,,,,,,,,,,,,
2018090905:313:1,2018-09-09T17:14:26.000Z,68.53,22.08,8.51,0.72,0.85,342.57,347.10,pass_arrived,71251.0,Patrick Chung,23.0,SS,75,home,left,NaN
2018090905:313:1,2018-09-09T17:14:26.000Z,58.67,40.16,4.92,2.55,0.50,23.22,10.87,pass_arrived,494287.0,Devin McCourty,32.0,FS,75,home,left,NaN
2018090905:313:1,2018-09-09T17:14:26.000Z,58.07,19.20,5.70,1.62,0.57,350.71,330.09,pass_arrived,2533057.0,Dont'a Hightower,54.0,MLB,75,home,left,NaN
2018090905:313:1,2018-09-09T17:14:26.000Z,70.59,3.03,1.55,0.63,0.16,48.19,254.13,pass_arrived,2533062.0,Stephon Gilmore,24.0,CB,75,home,left,NaN
2018090905:313:1,2018-09-09T17:14:26.000Z,60.76,16.34,4.23,2.10,0.43,22.74,17.47,pass_arrived,2541243.0,Duron Harmon,21.0,FS,75,home,left,NaN
2018090905:313:1,2018-09-09T17:14:26.000Z,93.09,45.67,6.40,3.26,0.65,345.15,357.52,pass_arrived,2543699.0,Kyle Van Noy,53.0,OLB,75,home,left,NaN
2018090905:313:1,2018-09-09T17:14:26.000Z,75.14,50.37,4.91,1.54,0.49,42.61,37.09,pass_arrived,2552255.0,Eric Rowe,25.0,CB,75,home,left,NaN
2018090905:313:1,2018-09-09T17:14:26.000Z,66.27,52.44,2.32,1.05,0.24,41.47,293.89,pass_arrived,2561143.0,Ja'Whaun Bentley,51.0,LB,75,home,left,NaN


In [37]:
ball_snap = week1[week1.index.isin([selected_play]) & (week1["event"]=="ball_snap")]["frameId"].values[0]
ball_snap

11

In [35]:
fpf_number = defense_fpf["frameId"].values[0]
fpf_number

67

In [36]:
fpa = defense_pa["frameId"].values[0]
fpa

75

Por cada jugador, ver el jugador más cercano rival y ver su distancia euclidea

In [33]:
def obtainMeanDistance(playerId,playId):
    idParts = playId.split(":")
    idWeek = idParts[2]
    week = weeks[int(idWeek)-1]
    print(playId)
    
    ball_snap = week[(week["id"]==playId) & (week["event"]=="ball_snap")]["frameId"].values[0]
    try:
        fpf_number = week[(week["id"]==playId) & (week["event"]=="pass_forward")]["frameId"].values[0]
    except:
        fpf_number = week[(week["id"]==playId) & (week["event"]=="pass_shovel")]["frameId"].values[0]
    #print(playId,ball_snap,fpf_number)
    mean_distances = []
    for frame in range(ball_snap,fpf_number+1):
        players = week[(week["frameId"]==frame)&(week["team"]==obtainOffenseTeam(playId))&(week["id"]==playId)]
        x = week[(week["frameId"]==frame)&(week["nflId"]==playerId)&(week["id"] == playId)]["x"].values[0]
        #x = week1[(week1["frameId"]==frame)&(week1["team"]==obtainOffenseTeam("2018090600:1037:1"))&(week1["route"].notnull())]
        y = week[(week["frameId"]==frame)&(week["nflId"]==playerId)&(week["id"]==playId)]["y"].values[0]
        #y = week1[(week1["playerId"]==playerId)]["y"].values[0]
        distances = []
        for index,row in players.iterrows():
            x_value = row["x"]
            y_value = row["y"]
            line = LineString([(x, y), (x_value, y_value)]) #distancia euclidea
            distances.append(line.length) #habría que tener en cuenta el jugador???
        #print(min(distances))
        mean_distances.append(min(distances))
    return sum(mean_distances)/len(mean_distances)
    

In [34]:
summary["meanDistance"] = summary.apply(lambda x: obtainMeanDistance(x["playerId"],x["playId"]),axis=1)

2018093006:2225:4
2018093006:2225:4
2018093006:2225:4
2018093006:2225:4
2018093006:2225:4
2018093006:2225:4
2018093006:2225:4
2018093006:2225:4
2018101500:3701:6
2018101500:3701:6
2018101500:3701:6
2018101500:3701:6
2018122300:3525:16
2018122300:3525:16
2018122300:3525:16
2018122300:3525:16
2018122300:3525:16
2018122300:3525:16
2018122300:3525:16
2018122300:3525:16
2018121700:852:15
2018121700:852:15
2018121700:852:15
2018121700:852:15
2018121700:852:15
2018121700:852:15
2018102500:273:8
2018102500:273:8
2018102500:273:8
2018102500:273:8
2018102500:273:8
2018102500:273:8
2018102500:273:8
2018102500:273:8
2018102500:273:8
2018111108:1321:10
2018111108:1321:10
2018111108:1321:10
2018111108:1321:10
2018111108:1321:10
2018101100:1536:6
2018101100:1536:6
2018101100:1536:6
2018101100:1536:6
2018101100:1536:6
2018101100:1536:6
2018101100:1536:6
2018121605:907:15
2018121605:907:15
2018121605:907:15
2018121605:907:15
2018091613:2893:2
2018091613:2893:2
2018091613:2893:2
2018091613:2893:2
201809

In [35]:
#save dataset
pickle_file = open('../resources/summary.pickle','wb')
pickle.dump(summary,pickle_file)
pickle_file.close()

In [109]:
def obtainMeanDistanceAfterPass(playerId,playId):
    idParts = playId.split(":")
    idWeek = idParts[2]
    week = weeks[int(idWeek)-1]
    print(playId)
    
    try:
        fpf_number = week[(week["id"]==playId) & (week["event"]=="pass_forward")]["frameId"].values[0]
    except:
        fpf_number = week[(week["id"]==playId) & (week["event"]=="pass_shovel")]["frameId"].values[0]
        
    try:
        fpa = week[(week["id"]==playId) & (week["event"]=="pass_arrived")]["frameId"].values[0]
    except:
        try:
            fpa = week[(week["id"]==playId) & (week["event"]=="pass_outcome_incomplete")]["frameId"].values[0]
        except:
            fpa = week[(week["id"]==playId) & (week["event"]=="pass_outcome_interception")]["frameId"].values[0]
    mean_distances = []
    for frame in range(fpf_number+1,fpa+1):
        players = week[(week["frameId"]==frame)&(week["team"]==obtainOffenseTeam(playId))&(week["id"]==playId)]
        x = week[(week["frameId"]==frame)&(week["nflId"]==playerId)&(week["id"] == playId)]["x"].values[0]
        #x = week1[(week1["frameId"]==frame)&(week1["team"]==obtainOffenseTeam("2018090600:1037:1"))&(week1["route"].notnull())]
        y = week[(week["frameId"]==frame)&(week["nflId"]==playerId)&(week["id"]==playId)]["y"].values[0]
        #y = week1[(week1["playerId"]==playerId)]["y"].values[0]
        distances = []
        for index,row in players.iterrows():
            x_value = row["x"]
            y_value = row["y"]
            line = LineString([(x, y), (x_value, y_value)]) #distancia euclidea
            distances.append(line.length) #habría que tener en cuenta el jugador???
        mean_distances.append(min(distances))
    return sum(mean_distances)/len(mean_distances)

In [ ]:
summary["meanDistanceAfterPass"] = summary.apply(lambda x: obtainMeanDistanceAfterPass(x["playerId"],x["playId"]),axis=1)

2018093006:2225:4
2018093006:2225:4
2018093006:2225:4
2018093006:2225:4
2018093006:2225:4
2018093006:2225:4
2018093006:2225:4
2018093006:2225:4
2018101500:3701:6
2018101500:3701:6
2018101500:3701:6
2018101500:3701:6
2018122300:3525:16
2018122300:3525:16


In [ ]:
summary

In [ ]:
#save dataset
pickle_file = open('../resources/summary.pickle','wb')
pickle.dump(summary,pickle_file)
pickle_file.close()

In [45]:
def obtainMeanPlayers(playerId,playId):
    idParts = playId.split(":")
    idWeek = idParts[2]
    week = weeks[int(idWeek)-1]
    
    try:
        fpa = week[(week["id"]==playId) & (week["event"]=="pass_arrived")]["frameId"].values[0]
    except:
        fpa = week[(week["id"]==playId) & (week["event"]=="pass_outcome_incomplete")]["frameId"].values[0]
    
    mean_number_players = []
    for frame in range(ball_snap,fpf_number+1):
        players = week[(week["frameId"]==frame)&(week["team"]==obtainOffenseTeam(playId))&(week["route"].notnull())&(week["id"]==playId)]
        x = week[(week["frameId"]==frame)&(week["nflId"]==playerId)&(week["id"] == playId)]["x"].values[0]
        y = week[(week["frameId"]==frame)&(week["nflId"]==playerId)&(week["id"]==playId)]["y"].values[0]
        p = Point(x, y)
        circle = p.buffer(5.0)
        number_players = 0
        for index,row in players.iterrows():
            x_value = row["x"]
            y_value = row["y"]
            point = Point(x_value,y_value)
            if circle.contains(point):
                number_players += 1
        mean_number_players.append(number_players)
    return sum(mean_number_players)/len(mean_number_players)

In [46]:
single_summary["numberPlayers5yds"] = single_summary.apply(lambda x: obtainMeanPlayers(x["playerId"],x["playId"]),axis=1)

In [49]:
def obtainMeanPlayersAfterPass(playerId):
    idParts = playId.split(":")
    idWeek = idParts[2]
    week = weeks[int(idWeek)-1]
    
    ball_snap = week[(week["id"]==playId) & (week["event"]=="ball_snap")]["frameId"].values[0]

    fpf_number = week[(week["id"]==playId) & (week["event"]=="pass_forward")]["frameId"].values[0]
    
    mean_number_players = []
    for frame in range(fpf_number+1,fpa+1):
        players = week1[(week1["frameId"]==frame)&(week1["team"]==obtainOffenseTeam(selected_play))&(week1["route"].notnull())&(week1.index.isin([selected_play]))]
        x = week1[(week1["frameId"]==frame)&(week1["nflId"]==playerId)&(week1.index.isin([selected_play]))]["x"].values[0]
        y = week1[(week1["frameId"]==frame)&(week1["nflId"]==playerId)&(week1.index.isin([selected_play]))]["y"].values[0]
        p = Point(x, y)
        circle = p.buffer(5.0)
        number_players = 0
        for index,row in players.iterrows():
            x_value = row["x"]
            y_value = row["y"]
            point = Point(x_value,y_value)
            if circle.contains(point):
                number_players += 1
        mean_number_players.append(number_players)
    return sum(mean_number_players)/len(mean_number_players)

In [50]:
single_summary["numberPlayers5ydsAfterPass"] = single_summary.apply(lambda x: obtainMeanPlayersAfterPass(x["playerId"],x["playId"]),axis=1)

In [51]:
single_summary

,playerId,meanDistance,meanDistanceAfterPass,numberPlayers5yds,numberPlayers5ydsAfterPass
id,,,,,
2018090905:313:1,71251.0,1.949711,2.831581,1.754386,1.0
2018090905:313:1,494287.0,13.515905,16.519959,0.000000,0.0
2018090905:313:1,2533057.0,2.793890,2.652474,0.859649,1.0
2018090905:313:1,2533062.0,3.299615,1.724586,1.000000,1.0
2018090905:313:1,2541243.0,8.609330,5.575882,0.228070,0.0
2018090905:313:1,2543699.0,13.628398,17.829817,0.333333,0.0
2018090905:313:1,2552255.0,2.598084,2.634874,0.824561,1.0
2018090905:313:1,2561143.0,2.976865,0.422223,0.754386,1.0


# Complex Features (c): Características dinámicas de la acción en general. Posiblemente podría ser cosas como variación de la superficie cubierta, variación de densidad, defensores en la zona del pase, .... 

In [52]:
def obtainAreaDefense(playId):
    defenders = defense_fpf.loc[playId]
    position_players = []
    playDirection = defenders["playDirection"].values[0]
    los = plays.loc[playId]["absoluteYardlineNumber"]
    
    for index,row in defenders.iterrows():
        if row["position"] in ["DL","NT","DT","DE"]:
            continue
        
        if playDirection == "right" and row["x"] < los:
            continue
        elif playDirection == "left" and row["x"] > los:
            continue
        
        position_players.append((row["x"],row["y"]))
        
    hull = ConvexHull(position_players)
    
    return hull.volume

In [53]:
single_summary["areaDifference"] = single_summary.index.map(obtainAreaDefense) - static_defense.loc[selected_play]["defenseArea"]

In [54]:
def calculateNumberDeepPlayers(playId):
    los = plays.loc[playId]["absoluteYardlineNumber"]
    defenders = defense_fpf.loc[playId]
    playDirection = defenders["playDirection"].values[0]
    deepzone = -1
    
    if playDirection == "right":
        deepzone = los + 15
    elif playDirection == "left":
        deepzone = los - 15
        
    number_players=0
    for index,row in defenders.iterrows():
        if row["position"] in ["DL","NT","DT","DE"]: #muy dificil que pase, pero por si acaso
            continue
        player_x = row["x"]
        if playDirection == "right" and player_x > deepzone:
            number_players+=1
        elif playDirection == "left" and player_x < deepzone:
            number_players+=1
            
    return number_players

In [55]:
single_summary["numberPlayersDeep"] = single_summary.index.map(calculateNumberDeepPlayers)

In [56]:
single_summary["numberPlayersDeepDifference"] = single_summary["numberPlayersDeep"] - static_defense.loc[selected_play]["deepzonePlayers"]

In [57]:
def calculateNumberUnderneath(playId):
    los = plays.loc[playId]["absoluteYardlineNumber"]
    defenders = defense_fpf.loc[playId]
    playDirection = defenders["playDirection"].values[0]
    deepzone = -1
    
    if playDirection == "right":
        deepzone = los + 15
    elif playDirection == "left":
        deepzone = los - 15
        
    number_players=0
    for index,row in defenders.iterrows():
        if row["position"] in ["DL","NT","DT","DE"]: #muy dificil que pase, pero por si acaso
            continue
        player_x = row["x"]
        if playDirection == "right" and player_x < deepzone and player_x > los:
            number_players+=1
        elif playDirection == "left" and player_x > deepzone and player_x < los:
            number_players+=1
            
    return number_players

In [58]:
single_summary["numberPlayersUnderneath"] = single_summary.index.map(calculateNumberUnderneath)

In [59]:
single_summary["densityDifference"] = (single_summary["numberPlayersDeep"] + single_summary["numberPlayersUnderneath"])/single_summary["areaDifference"]

In [60]:
def calculateHeight(playId):
    defenders = defense_fpf.loc[playId]
    playDirection = defenders["playDirection"].values[0]
    los = plays.loc[playId]["absoluteYardlineNumber"]
    
    players_x = []
    
    for index,row in defenders.iterrows():
        if playDirection == "right" and los < row["x"]:
            players_x.append(row["x"])
        elif playDirection == "left" and los > row["x"]:
            players_x.append(row["x"])
        
    return abs(max(players_x) - min(players_x))

In [61]:
single_summary["heightDifference"] = single_summary.index.map(calculateHeight) - static_defense.loc[selected_play]["height"]

In [62]:
def calculateWidth(playId):
    defenders = defense_fpf.loc[playId]
    playDirection = defenders["playDirection"].values[0]
    los = plays.loc[playId]["absoluteYardlineNumber"]
    
    players_y = []
    
    for index,row in defenders.iterrows():
        if playDirection == "right" and los < row["x"]:
            players_y.append(row["y"])
        elif playDirection == "left" and los > row["x"]:
            players_y.append(row["y"])
        
    return max(players_y) - min(players_y)

In [63]:
single_summary["widthDifference"] = single_summary.index.map(calculateWidth) - static_defense.loc[selected_play]["width"]

# se pueden añadir las características estáticas del comienzo o el cluster.

In [91]:
clusters = pd.read_csv("../unsupervised_datasets/kmeans/static_defenses_rs_kmeans_without_outliers.csv",index_col="id")
clusters.head()

,defensivelinezonePlayers,deepzonePlayers,hookzonePlayers,curlzonePlayers,flatzonePlayers,defenseArea,defenseAreaCoverDefenders,width,height,numberQBs,numberWRs,numberTEs,numberRBs,numberFBs,numberOffensivePlayersAnotherPosition,numberSafeties,numberLBs,numberCBs,strongSide,numberPlayersDefenseStrongSide,numberPlayersDefenseWeakSide,numberPlayersOffenseStrongSide,numberPlayersOffenseWeakSide,differenceOffenseVsDefenseWidth,differenceOffenseVsDefenseStrongSide,differenceOffenseVsDefenseWeakSide,HeightByWeightDeep,HeightByWeightHook,HeightByWeightCurl,HeightByWeightFlat,WeightByArea,density,densityNoLine,densityInsidePoints,densityInsidePointsNoLine,cluster
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2018090600:75:1,0,1,3,2,1,194.31095,194.31095,22.96,15.25,1,2,1,1,1,0,3,2,2,1,3,4,3,2,-0.63,0,-2,0.020227,0.018643,0.020066,0.021034,3.376032,0.036025,0.036025,6.974268,6.974268,0
2018090600:146:1,0,1,3,2,1,204.17725,204.17725,29.22,12.57,1,3,1,1,0,0,2,2,3,0,3,4,2,3,-1.15,-1,-1,0.020227,0.019228,0.020356,0.020455,3.178611,0.034284,0.034284,6.975511,6.975511,0
2018090600:168:1,0,1,2,2,2,277.65495,277.65495,33.67,15.21,1,2,1,2,0,0,2,2,3,1,5,2,4,1,0.71,-1,-1,0.020227,0.017683,0.020997,0.020745,2.337434,0.025211,0.025211,6.985594,6.985594,0
2018090600:190:1,1,0,2,3,1,189.16960,165.05520,26.42,12.91,1,3,1,1,0,0,3,1,3,0,2,5,2,3,-3.11,0,-2,0.000000,0.020120,0.021193,0.020455,3.192871,0.037004,0.036351,6.973569,6.969707,0
2018090600:320:1,1,0,3,3,1,187.12510,153.20020,27.80,13.29,1,3,1,1,0,0,2,3,3,1,3,5,3,2,-4.82,0,-3,0.000000,0.018995,0.019537,0.021279,4.281979,0.042752,0.045692,7.978624,7.980418,0


In [92]:
clusters["cluster"].unique()

array([0, 1, 2])

In [68]:
single_summary["cluster"] = clusters.loc[selected_play]["cluster"]

# Game context features(g): down, score ...

In [77]:
single_summary["homeScore"] = plays.loc[selected_play]["preSnapHomeScore"]

In [78]:
single_summary["visitorScore"] = plays.loc[selected_play]["preSnapVisitorScore"]

In [79]:
single_summary["down"] = plays.loc[selected_play]["down"]

In [80]:
single_summary["quarter"] = plays.loc[selected_play]["quarter"]

In [81]:
single_summary["yardsToGo"] = plays.loc[selected_play]["yardsToGo"]

In [82]:
def obtainYardsToEndZone(playId):
    absoluteYard = plays.loc[selected_play]["absoluteYardlineNumber"]
    direction = defense.loc[selected_play]["playDirection"][0]
    if direction == "right":
        value = 120 - absoluteYard
    elif direction == "left":
        value = absoluteYard - 10
        
    return value

In [83]:
single_summary["yardsToEndZone"] = single_summary.index.map(obtainYardsToEndZone)

In [84]:
single_summary["gameClock"] = plays.loc[selected_play]["gameClock"]

In [85]:
single_summary

,playerId,meanDistance,meanDistanceAfterPass,numberPlayers5yds,numberPlayers5ydsAfterPass,areaDifference,numberPlayersDeep,numberPlayersDeepDifference,numberPlayersUnderneath,densityDifference,heightDifference,widthDifference,cluster,homeScore,visitorScore,down,quarter,yardsToGo,yardsToEndZone,gameClock
id,,,,,,,,,,,,,,,,,,,,
2018090905:313:1,71251.0,1.949711,2.831581,1.754386,1.0,224.9914,3,3,4,0.031112,3.18,14.08,0.0,7.0,0.0,2,1,9,72.0,11:07:00
2018090905:313:1,494287.0,13.515905,16.519959,0.000000,0.0,224.9914,3,3,4,0.031112,3.18,14.08,0.0,7.0,0.0,2,1,9,72.0,11:07:00
2018090905:313:1,2533057.0,2.793890,2.652474,0.859649,1.0,224.9914,3,3,4,0.031112,3.18,14.08,0.0,7.0,0.0,2,1,9,72.0,11:07:00
2018090905:313:1,2533062.0,3.299615,1.724586,1.000000,1.0,224.9914,3,3,4,0.031112,3.18,14.08,0.0,7.0,0.0,2,1,9,72.0,11:07:00
2018090905:313:1,2541243.0,8.609330,5.575882,0.228070,0.0,224.9914,3,3,4,0.031112,3.18,14.08,0.0,7.0,0.0,2,1,9,72.0,11:07:00
2018090905:313:1,2543699.0,13.628398,17.829817,0.333333,0.0,224.9914,3,3,4,0.031112,3.18,14.08,0.0,7.0,0.0,2,1,9,72.0,11:07:00
2018090905:313:1,2552255.0,2.598084,2.634874,0.824561,1.0,224.9914,3,3,4,0.031112,3.18,14.08,0.0,7.0,0.0,2,1,9,72.0,11:07:00
2018090905:313:1,2561143.0,2.976865,0.422223,0.754386,1.0,224.9914,3,3,4,0.031112,3.18,14.08,0.0,7.0,0.0,2,1,9,72.0,11:07:00


In [93]:
single_summary.to_csv("single_summary_20180905_313_1.csv")

In [4]:
df = pd.read_csv("single_summary_20180905_313_1.csv",index_col=0)
df

,playerId,meanDistance,meanDistanceAfterPass,numberPlayers5yds,numberPlayers5ydsAfterPass,areaDifference,numberPlayersDeep,numberPlayersDeepDifference,numberPlayersUnderneath,densityDifference,heightDifference,widthDifference,cluster,homeScore,visitorScore,down,quarter,yardsToGo,yardsToEndZone,gameClock
id,,,,,,,,,,,,,,,,,,,,
2018090905:313:1,71251.0,1.949711,2.831581,1.754386,1.0,224.9914,3,3,4,0.031112,3.18,14.08,0.0,7.0,0.0,2,1,9,72.0,11:07:00
2018090905:313:1,494287.0,13.515905,16.519959,0.000000,0.0,224.9914,3,3,4,0.031112,3.18,14.08,0.0,7.0,0.0,2,1,9,72.0,11:07:00
2018090905:313:1,2533057.0,2.793890,2.652474,0.859649,1.0,224.9914,3,3,4,0.031112,3.18,14.08,0.0,7.0,0.0,2,1,9,72.0,11:07:00
2018090905:313:1,2533062.0,3.299615,1.724586,1.000000,1.0,224.9914,3,3,4,0.031112,3.18,14.08,0.0,7.0,0.0,2,1,9,72.0,11:07:00
2018090905:313:1,2541243.0,8.609330,5.575882,0.228070,0.0,224.9914,3,3,4,0.031112,3.18,14.08,0.0,7.0,0.0,2,1,9,72.0,11:07:00
2018090905:313:1,2543699.0,13.628398,17.829817,0.333333,0.0,224.9914,3,3,4,0.031112,3.18,14.08,0.0,7.0,0.0,2,1,9,72.0,11:07:00
2018090905:313:1,2552255.0,2.598084,2.634874,0.824561,1.0,224.9914,3,3,4,0.031112,3.18,14.08,0.0,7.0,0.0,2,1,9,72.0,11:07:00
2018090905:313:1,2561143.0,2.976865,0.422223,0.754386,1.0,224.9914,3,3,4,0.031112,3.18,14.08,0.0,7.0,0.0,2,1,9,72.0,11:07:00


Tres etiquetas:
- mala: no ha defendido bien
- buena: ha defendido bien en toda la jugada
- regular: ha defendido bien un tramo pero ha perdido la marca

In [6]:
valorations = ["regular","regular","regular","buena","mala","mala","mala","buena"] #para mi el mejor es Ja'Whaun Bentley

In [7]:
df["subjective_evaluation"] = valorations

In [8]:
df

,playerId,meanDistance,meanDistanceAfterPass,numberPlayers5yds,numberPlayers5ydsAfterPass,areaDifference,numberPlayersDeep,numberPlayersDeepDifference,numberPlayersUnderneath,densityDifference,...,widthDifference,cluster,homeScore,visitorScore,down,quarter,yardsToGo,yardsToEndZone,gameClock,subjective_evaluation
id,,,,,,,,,,,,,,,,,,,,,
2018090905:313:1,71251.0,1.949711,2.831581,1.754386,1.0,224.9914,3,3,4,0.031112,...,14.08,0.0,7.0,0.0,2,1,9,72.0,11:07:00,regular
2018090905:313:1,494287.0,13.515905,16.519959,0.000000,0.0,224.9914,3,3,4,0.031112,...,14.08,0.0,7.0,0.0,2,1,9,72.0,11:07:00,regular
2018090905:313:1,2533057.0,2.793890,2.652474,0.859649,1.0,224.9914,3,3,4,0.031112,...,14.08,0.0,7.0,0.0,2,1,9,72.0,11:07:00,regular
2018090905:313:1,2533062.0,3.299615,1.724586,1.000000,1.0,224.9914,3,3,4,0.031112,...,14.08,0.0,7.0,0.0,2,1,9,72.0,11:07:00,buena
2018090905:313:1,2541243.0,8.609330,5.575882,0.228070,0.0,224.9914,3,3,4,0.031112,...,14.08,0.0,7.0,0.0,2,1,9,72.0,11:07:00,mala
2018090905:313:1,2543699.0,13.628398,17.829817,0.333333,0.0,224.9914,3,3,4,0.031112,...,14.08,0.0,7.0,0.0,2,1,9,72.0,11:07:00,mala
2018090905:313:1,2552255.0,2.598084,2.634874,0.824561,1.0,224.9914,3,3,4,0.031112,...,14.08,0.0,7.0,0.0,2,1,9,72.0,11:07:00,mala
2018090905:313:1,2561143.0,2.976865,0.422223,0.754386,1.0,224.9914,3,3,4,0.031112,...,14.08,0.0,7.0,0.0,2,1,9,72.0,11:07:00,buena
